# **Transcription with Conversation-Level Sentiment Annotations**
Below, we use the Hume and GPT APIs to generate conversation-level sentiment annotations for a Zoom conversation.

> We design a two-part pipeline to visualize Zoom meetings with conversation-level sentiment annotations. We first introduce novel metrics to capture conversation-level sentiments along three axes: comprehension, consensus, and cordiality. To obtain these metrics, we first identify each speaker's individual expressed sentiments during each of their responses. To determine speaker sentiment, we segment Zoom recordings by speaker and feed the video data, audio file (including information on voice prosity), and transcript (text content) of each segment to an off-the-shelf model that outputs a quantitative measure of the extent to which the speaker expresses 48 emotions. Afterward, for each segment, we combine the speaker's top 5 emotions with weights, uniformly sampled facial expressions, and spoken words in an instruction-tuned prompt to a multimodal large language model in order to determine conversation-level metrics.



# Initialization

In [1]:
# Install libraries
!pip install hume
!pip install hume[stream]
!pip install openai
!pip install python-dotenv
!pip install pydub
!pip install ffmpeg
!pip install moviepy webvtt-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=171a43f6414cb19953569938ff687616f058f54c8d46c86f57889f86687fa2a8
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl siz

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests
import base64
from pydub import AudioSegment
from hume import HumeBatchClient
import json
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import userdata
import webvtt
from moviepy.editor import VideoFileClip
from google.colab import userdata

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_provider" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_resource" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
os.environ['HUME_API_KEY'] = userdata.get('HUME_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [4]:
# Initialize Hume and OpenAI clients
hume_client = HumeBatchClient(os.getenv('HUME_API_KEY'))
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Dataset Preparation

<!-- ## First prepare the recording for one person

Below is a set of sentences using different semantic tones

**Encouraging**: "I'm confident that if we all pull together, we can complete the project ahead of the deadline. Let’s keep up the great work!"

**Urgent**: "It’s crucial that we meet the project deadline. Every moment counts, so let’s prioritize efficiently!"

**Casual**: "Just a heads-up, we gotta wrap this project up by the due date. Let's get it done and have some fun along the way!"

**Formal**: "It is imperative that we adhere to the stipulated timeline for the completion of this project. Your diligent attention to the deadlines is greatly appreciated."

**Optimistic**: "With the progress we’re making, I’m sure we’ll finish the project well before the deadline. Keep up the fantastic effort!"

**Skeptical**: "Considering our current pace, are we sure we can finish the project by the deadline? We might need to reassess our strategy."

**Direct**: "Finish the project by the deadline. No exceptions." -->

<!-- # Get Inputs
load video from data folder, they are two processed Zoom recording files, with role1.mp4 and role2.mp4
the transcript file is transcript.vtt -->

## **Step1: Segment the video by speaker**

Zoom already segments the VTT file into clips (segments in which someone is continuously speaking). We combine adjacent clips with the same speaker and then segment the associated video file.


In [5]:
# Define file locations
vtt_file = './data/zoom/clipped.vtt'
video_file = './data/zoom/video.mp4'

# Define the output directory
output_dir = './data/zoom_clipped/'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

In [6]:
def parse_vtt(vtt_file):
  '''
  @param vtt_file: a VTT file
  @returns a JSON that maps names of speakers to clips, where each clip is represented as an array of strings of form [start_timestamp, end_timestamp, text_said]
  '''
  user_clips = {}
  last_speaker = None
  for caption in webvtt.read(vtt_file):
      if ':' in caption.text:
          username, text = caption.text.split(':', 1)
          username = username.strip().replace(' ', '-')
          text = text.strip()
          if username not in user_clips:
              user_clips[username] = []
          if username == last_speaker:
            user_clips[username][-1] = (user_clips[username][-1][0], caption.end, user_clips[username][-1][2] + " " + text)
          else:
            user_clips[username].append((caption.start, caption.end, text))

          last_speaker = username
  return user_clips

def process_clips(video_file, user_clips):
    video = VideoFileClip(video_file)
    for user, entries in user_clips.items():
        for start, end, text in entries:
            try:
                # Format the filename base
                filename_base = f"{user}_{start.replace(':', '-')}_{end.replace(':', '-')}"
                video_filename = f"{output_dir}{filename_base}.mp4"
                text_filename = f"{output_dir}{filename_base}.txt"
                audio_filename = f"{output_dir}{filename_base}.mp3"

                # Process video
                clip = video.subclip(start, end)
                clip.write_videofile(video_filename, codec="libx264")

                # Save transcript text
                with open(text_filename, 'w') as text_file:
                    text_file.write(text)

                # Extract and save audio
                audio = clip.audio
                audio.write_audiofile(audio_filename)

            except Exception as e:
                print(f"Failed to process {filename_base}: {e}")

# Parse VTT and process clips
user_clips = parse_vtt(vtt_file)
process_clips(video_file, user_clips)

FileNotFoundError: [Errno 2] No such file or directory: './data/zoom/clipped.vtt'

## **Batching for Hume**


In [ ]:
# Document https://humeai.github.io/hume-python-sdk/0.2.0/
from hume import HumeBatchClient

# from hume.models.config import BurstConfig # for burst
from hume.models.config import FaceConfig # for face expression
from hume.models.config import LanguageConfig # for text and audio contents
from hume.models.config import ProsodyConfig # for audio prosody(tone)

# Define configurations for Hume client
input_dir = './data/zoom_clipped/'
output_dir = './data/outputs/hume/'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

def process_file(filepath, config, suffix):
    filename = os.path.basename(filepath)
    file_extension = os.path.splitext(filename)[1]

    # Submit the job
    job = hume_client.submit_job(None, [config], files=[filepath])
    print(f"Processing {filename} with {config.__class__.__name__}...")

    # Await and retrieve results
    job.await_complete()
    predictions = job.get_predictions()
    print(predictions)

    # Form the output filename and save the predictions
    output_filename = filename.replace(file_extension, suffix + '.json')
    output_filepath = os.path.join(output_dir, output_filename)
    job.download_predictions(output_filepath)
    print(f"Predictions for {filename} downloaded to {output_filepath}")

def process_files(input_dir):
    # List all files in the directory
    files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

    # Process each file with the appropriate configurations
    for filepath in files:
        file_extension = os.path.splitext(filepath)[1]

        # Apply different configurations based on file type
        if file_extension in ['.mp4', '.mov']:  # Video files
            # Face analysis on video
            process_file(filepath, FaceConfig(), '_face')
            # Language analysis on video, it outputs the language for the whole sentense, different from the text_language analysis
            # process_file(filepath, LanguageConfig(), '_video_lang')
            # Prosody analysis on video. it is the same as audio prosody, so we don't need it
            # process_file(filepath, ProsodyConfig(), '_video_prosody')
        elif file_extension == '.mp3':  # Audio files
            # Prosody analysis on audio, audio prosody outputs the tone analysis for the whole period
            process_file(filepath, ProsodyConfig(), '_prosody')
            # Language analysis on audio, it is essentially the same as text language analysis, so we don't need it
            # process_file(filepath, LanguageConfig(), '_audio_lang')
        elif file_extension == '.txt':  # Text files
            # Language analysis on text, it's word by word, too detail, we don't need it now
            process_file(filepath, LanguageConfig(), '_lang')
            # # Prosody analysis on text, after test, text doesn't have prosody outputs
            # process_file(filepath, ProsodyConfig(), '_text_prosody')

# Process all files in the input directory
process_files(input_dir)

##**Individual Sentiment Annotations**

We call the Hume's prosody, facial expression, and language APIs on the clips we created above and isolate the top 5 emotions expressed by each speaker during each line.

#### **Helper Functions**

In [ ]:
def predict_sentiment(modality):
  '''
  Predicts the sentiment of all clips based on the specified modality.
  @param modality: String suffix describing modality in files, i.e. "face," "prosody," or "language"
  @return Dataframe of clip timing, predicted emotions, top k emotions, confidence, and additional columns depending on the modality
  '''
  # Path to the directory containing the files
  directory_path = './data/outputs/hume/'

  # List all files in the directory with the relevant suffix
  if modality == "language":
    suffix = "_lang"
  else:
    suffix = "_" + modality
  relevant_files = [f for f in os.listdir(directory_path) if suffix in f and f.endswith('.json')]

  # List to hold all the dataframes
  dataframes = []

  # Count errors
  errors = 0

  # Iterate over each file and process similarly to the first file
  for file_name in relevant_files:
      file_path = os.path.join(directory_path, file_name)
      with open(file_path, 'r') as file:
          data = json.load(file)
          # TODO: replace with corruption checking. Removes clips with errors/file not recognized
          if data[0]['results']['errors']:
            errors += 1
            continue
          predictions = data[0]['results']['predictions'][0]['models'][modality]['grouped_predictions'][0]['predictions']
          df = pd.DataFrame(predictions)
          dataframes.append(df)

  # Combine all dataframes into one
  combined_df = pd.concat(dataframes, ignore_index=True)

  # extract top emotions
  df_full = top_emotions_to_df(combined_df)

  # display
  print("Omitted clips: ", errors)
  display(df_full.head())
  return df_full


In [ ]:
def top_emotions(emotion_list, k=5):
  '''
  @params: takes in a list of objects with properties 'name' (String, name of emotion) and 'score' (float, degree of expression)
  @returns a list containing the k emotions with the highest scores, in the same object format as the input
  '''
  return sorted(emotion_list, key=lambda x: x['score'], reverse=True)[:k]

In [ ]:
def top_emotions_to_df(df):
  '''
  @params: takes in a dataframe with a column labeled "emotions"
  @returns: the same dataframe with an additional column labeled "top emotions" that has the top 5 emotions at that time stamp
  '''
  df_ranked = df['emotions'].apply(top_emotions)
  df_ranked = df_ranked.rename('top emotions')
  df_full = df.join(df_ranked)
  return df_full

In [ ]:
def plot_emotions(df, i, title="Emotions Expressed"):
  '''
  Displays a bar chart of the top k emotions experienced during interaction number i
  @param df: dataframe with a column 'top emotions' that has highest-scoring emotions experienced per timestamp
  @param i: index of the interaction to plot
  @returns: None
  '''
  emotions = df['top emotions'][i]

  # Plot the emotions
  plt.figure()
  plt.bar([emotion['name'] for emotion in emotions], [emotion['score'] for emotion in emotions])
  plt.xlabel("Emotion")
  plt.ylabel("Intensity")
  plt.title(title)
  plt.show()

In [ ]:
def plot_multiple_emotions(df_list, i_list, titles):
    '''
    Displays bar charts for the top k emotions experienced during specified interactions, organized in subplots.
    @param df_list: list of dataframes, each with a column 'top emotions' that has highest-scoring emotions experienced per timestamp
    @param i_list: list of indices for the interactions to plot (same order as dataframes)
    @param titles: list of titles for each subplot
    @returns: None
    '''
    num_plots = len(df_list)
    fig, axs = plt.subplots(1, num_plots, figsize=(6 * num_plots, 5), sharey=True)

    for ax, df, i, title in zip(axs, df_list, i_list, titles):
        emotions = df['top emotions'][i]
        ax.bar([emotion['name'] for emotion in emotions], [emotion['score'] for emotion in emotions])
        ax.set_xlabel("Emotion")
        ax.set_title(title)

    axs[0].set_ylabel("Intensity")

    plt.tight_layout()
    plt.show()

#### **Main**


In [ ]:
prosody = predict_sentiment("prosody")
face = predict_sentiment("face")
language = predict_sentiment("language")

In [ ]:
df_list = [prosody, face, language]
i_list = [0, 0, 0]
titles = [
    "Emotions Expressed Based on Prosody",
    "Emotions in First Utterance Based on Facial Expression",
    "Emotions in First Utterance Based on Spoken Word"
]

plot_multiple_emotions(df_list, i_list, titles)

## zip and download the data folder

In [ ]:
import os
import zipfile
from google.colab import files as colab_files  # Renaming the import to avoid conflicts

def make_zip(output_filename, source_dir):
    with zipfile.ZipFile(output_filename + '.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                # Store files relative to the source directory in the zip file
                zipf.write(file_path, os.path.relpath(file_path, start=source_dir))
    colab_files.download(output_filename + '.zip')  # Using the renamed import

# Example usage: zip the contents of './data/' into 'data.zip'
make_zip('data', './data/')


## **Multimodal Fusion**
We've identified the top 5 emotions expressed through each of the three modes: facial expression, vocal prosody, and spoken word. We try several methods to combine our analysis:


*   Passing all emotions and scores to GPT-4V
*   Weighting all scores by the confidence/probability value given, and selecting the top 5 emotions expressed across all three modalities.



# **ChatGPT-4V**

##**Prompt Generation**

Improvement for next time: clarify that GPT can and should use decimals; be stricter on the output

The full prompt text:

> You're an expert at understanding the social dynamics underlying conversations. In particular, you are great at monitoring three conversational metrics: comprehension, consensus, and cordiality. You produce quantitative measure of each of these metrics by averaging how each participant would rate the other on a 5-point Likert scale, with 1 = low and 5 = high. Cordiality is the extent to which each participant would rate herself as 'liking' the other person. Consensus is the extent to which each participant would rate herself as 'agreeing' with the other person. Confusion is the extent to which each participant would rate herself as 'understanding' what the other person is saying. You characterize conversations by considering each participant's facial expression, dialogue, and expressed emotions (based on vocal prosody, facial expression, and language), which will be provided to you. I will now provide you with a series of interactions. For each interaction, please reply in exactly this format (excluding quotation marks): '[CORDIALITY_SCORE],[CONSENSUS_SCORE],[CONFUSION_SCORE]' and replace the variables, which have format [VARIABLE_NAME], with your 1-5 rating.

In [ ]:
def generate_prompt_df(df, modality):
  def generate_prompt(top_emotions, modality):
    '''
    Generates a GPT-4 prompt that incorporates the individual sentiments identified above
    '''
    prompt = f"Judging by their {modality}, the speaker expressed the following emotions: "
    for emotion in top_emotions:
      prompt += f"{emotion['name']}, with an intensity of {emotion['score']}, "
    prompt = prompt[:-2] + "." # fix punctuation
    return prompt
  prompts = df['top emotions'].apply(generate_prompt, modality=modality)
  prompts = prompts.rename('prompt')
  df_full = df.join(prompts)
  return df_full

In [ ]:
face_with_prompt = generate_prompt_df(face, "face")
lang_with_prompt = generate_prompt_df(language, "language")
prosody_with_prompt = generate_prompt_df(prosody, "prosody")

In [ ]:
def generate_conversation_metrics_df(df):
  def generate_conversation_metric(prompt):
    instructions = "You're an expert at understanding the social dynamics underlying conversations. In particular, you are great at monitoring three conversational metrics: comprehension, consensus, and cordiality. You produce quantitative measure of each of these metrics by averaging how each participant would rate the other on a 5-point Likert scale, with 1 = low and 5 = high. Cordiality is the extent to which each participant would rate herself as 'liking' the other person. Consensus is the extent to which each participant would rate herself as 'agreeing' with the other person. Confusion is the extent to which each participant would rate herself as 'understanding' what the other person is saying. You characterize conversations by considering each participant's facial expression, dialogue, and expressed emotions (based on vocal prosody, facial expression, and language), which will be provided to you. I will now provide you with a series of interactions. For each interaction, please reply in exactly this format (excluding quotation marks): '[CORDIALITY_SCORE],[CONSENSUS_SCORE],[CONFUSION_SCORE]' and replace the variables, which have format [VARIABLE_NAME], with your 1-5 rating."
    full_prompt = instructions + prompt
    message=[{"role": "assistant", "content": full_prompt}]
    temperature=0.2
    max_tokens=256
    frequency_penalty=0.0

    response = openai_client.chat.completions.create(
        model="gpt-4",
        messages = message,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty
    )
    return response.choices[0].message.content

  conv_metrics = df['prompt'].apply(generate_conversation_metric)
  conv_metrics = conv_metrics.rename('conv_metrics')
  df_full = df.join(conv_metrics)
  return df_full

In [ ]:
pros = generate_conversation_metrics_df(prosody_with_prompt)

### **Let's observe how the conversational metrics change over time.**



In [ ]:
conv_metrics = pros['conv_metrics'].values.tolist()

**We have some issues with GPT deviating from the output script, either due to a prompting mistake or because GPT switches between using ' and " as quotation marks. Check out part of the output below.**

In [ ]:
conv_metrics

**Let's clean up the conversation metrics and convert them to lists so they're easier to plot.**

In [ ]:
import re

pattern = pattern = r'(\d+\.?\d*),(\d+\.?\d*),(\d+\.?\d*)'
def parse_and_convert(strings):
    pattern = re.compile(r'(\d+\.?\d*),(\d+\.?\d*),(\d+\.?\d*)')
    results = []

    for string in strings:
        matches = pattern.search(string)
        if matches:
            numbers = [float(num) for num in matches.groups()]
            results.append(numbers)

    return results

In [ ]:
parsed_data = parse_and_convert(conv_metrics)

In [ ]:
time_points = range(len(parsed_data))
values1, values2, values3 = zip(*parsed_data)


fig, ax = plt.subplots()


ax.plot(time_points, values1, label='Cordiality', marker='o')
ax.plot(time_points, values2, label='Consensus', marker='o')
ax.plot(time_points, values3, label='Confusion', marker='o')

ax.set_xlabel('Time Point')
ax.set_ylabel('Value')
ax.set_title('Values Over Time')
ax.legend()

##**Ideas for Improvement**


*   Few shot learning by feeding GPT some very short clips (likely to run into rate limit issues)
*   Instead of prompting on each speaker's response, feed in multiple exchanges so that GPT has context on the conversation
*   Chain of thought prompting (i.e. output what one participant would rate the other and explain why, then output what the other participant would rate the other and explain why, then give final answer in a particular (parsable) format
*   Make the prompt a lot shorter/cut out unnecessary words and pronouns



# **End of Notebook**

In [ ]:
# convert video frames to images


In [ ]:
# Function to encode an image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Encode the image and prepare for upload
image_path = "./data/david_hume.jpeg"
base64_image = encode_image(image_path)


In [ ]:
# Using OpenAI's GPT to understand image contents
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}",
}

payload = {
    "model": "gpt-4-turbo",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ],
        },
    ],
    "max_tokens": 300,
}

response = requests.post(
    "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
)

print(response.json())

# limit the word count
# JSON mode
# does it maintain attention for each post
# rubric and scale
# one slider for conversation going on well
# judge the conversation



In [ ]:
# Transcribe audio using OpenAI's Whisper
def transcribe(audio_path):
    with open(audio_path, "rb") as audio_file:
        transcription = openai_client.audio.transcriptions.create(
            model="whisper-1", file=audio_file, response_format="text"
        )
        return transcription

audio_path = "segment_1.mp3"
transcription = transcribe(audio_path)
print(transcription)


In [ ]:
# Segment a longer audio into manageable parts
def segment_audio(audio_path, segment_duration_ms):
    song = AudioSegment.from_mp3(audio_path)
    segments = []
    for i in range(0, len(song), segment_duration_ms):
        segment = song[i:i+segment_duration_ms]
        segment_path = f"segment_{i//segment_duration_ms}.mp3"
        segment.export(segment_path, format="mp3")
        segments.append(segment_path)
    return segments

# Example usage
segmented_audio_paths = segment_audio("long_audio.mp3", 10 * 60 * 1000)  # 10 minutes in ms
print("Segmented audio into:", segmented_audio_paths)
